## Extreme Gradient Boosting

In [1]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV

import data_processing
import feature_extraction

In [2]:
df = data_processing.read_all_users('Geolife_data')

[1/38] processing user User37
[2/38] processing user User30
[3/38] processing user User31
[4/38] processing user User36
[5/38] processing user User38
[6/38] processing user User2
[7/38] processing user User5
[8/38] processing user User4
[9/38] processing user User3
[10/38] processing user User25
[11/38] processing user User22
[12/38] processing user User14
[13/38] processing user User13
[14/38] processing user User12
[15/38] processing user User15
[16/38] processing user User23
[17/38] processing user User24
[18/38] processing user User33
[19/38] processing user User34
[20/38] processing user User35
[21/38] processing user User32
[22/38] processing user User8
[23/38] processing user User6
[24/38] processing user User1
[25/38] processing user User7
[26/38] processing user User9
[27/38] processing user User21
[28/38] processing user User26
[29/38] processing user User19
[30/38] processing user User10
[31/38] processing user User17
[32/38] processing user User28
[33/38] processing user Us

In [3]:
df=data_processing.process_classes(df)

In [4]:
df=data_processing.calculations(df)
df

,lat,lon,alt,elapsed time,label,time,trajectory,user,distance,speed,time_diff,acceleration,bearing,heading_change,Vrate,angular_velocity,angular_acceleration
0,39.894178,116.318200,-777.0,39535.621296,train,2008-03-28 14:54:40,01020080328144824.plt,33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,39.894505,116.321132,-777.0,39535.621690,train,2008-03-28 14:55:14,01020080328144824.plt,33,0.252764,0.007434,34.0,NaN,81.728206,NaN,NaN,NaN,NaN
2,39.894953,116.326452,-777.0,39535.622373,train,2008-03-28 14:56:13,01020080328144824.plt,33,0.456583,0.007739,59.0,5.160437e-06,83.734582,2.006376e+00,3.934334e-02,3.400637e-02,NaN
3,39.894600,116.332542,-777.0,39535.623056,train,2008-03-28 14:57:12,01020080328144824.plt,33,0.521027,0.008831,59.0,1.851318e-05,94.318545,1.058396e+01,1.236873e-01,1.793892e-01,2.464116e-03
4,39.889622,116.337040,-777.0,39535.623738,train,2008-03-28 14:58:11,01020080328144824.plt,33,0.673539,0.011416,59.0,4.381261e-05,145.266086,5.094754e+01,2.264335e-01,8.635176e-01,1.159540e-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
471,40.029320,116.411975,289.0,39781.103808,train,2008-11-29 02:29:29,17920081129015805.plt,36,0.023240,0.011620,2.0,-2.799201e-05,180.419823,2.068969e-01,4.817827e-03,-1.034485e-01,5.147249e-02
472,40.029111,116.411963,275.0,39781.103831,train,2008-11-29 02:29:31,17920081129015805.plt,36,0.023262,0.011631,2.0,5.456289e-06,182.517368,2.097545e+00,9.382246e-04,1.048773e+00,5.761106e-01
473,40.028904,116.411962,274.0,39781.103854,train,2008-11-29 02:29:33,17920081129015805.plt,36,0.023018,0.011509,2.0,-6.117035e-05,180.211944,2.305425e+00,1.063023e-02,-1.152712e+00,-1.100742e+00
474,40.028697,116.411961,274.0,39781.103877,train,2008-11-29 02:29:35,17920081129015805.plt,36,0.023018,0.011509,2.0,-1.147129e-13,180.211944,6.431768e-07,1.993490e-11,3.215884e-07,5.763563e-01


In [5]:
feature_df = df.groupby('trajectory').apply(feature_extraction.extract_features).reset_index(drop=True)

/var/folders/6q/9kdss80974ldrx7hy_83wyn00000gn/T/ipykernel_35050/544797563.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  feature_df = df.groupby('trajectory').apply(feature_extraction.extract_features).reset_index(drop=True)


In [6]:
feature_df

,label,trajectory,start_time,trajectory_length,total_distance,average_speed,max_speed,duration,average_acceleration,speed_std,acceleration_std,bearing_changes,HCR,stops,SR,VC,VCR
0,train,01020080328144824.plt,14,69,62.604343,0.015478,0.031892,1.072778,6.854169e-07,0.012885,0.000053,10,0.144928,18,0.260870,15,0.217391
1,car,01020080617094444.plt,12,370,66.132033,0.008436,0.016202,0.113889,-3.304812e-06,0.004180,0.001117,30,0.081081,1,0.002703,55,0.148649
2,train,01020080618121033.plt,15,1509,142.036173,0.019471,0.034116,0.496944,2.207413e-05,0.007687,0.000693,10,0.006627,0,0.000000,17,0.011266
3,train,01020080618160000.plt,0,9502,728.675081,0.022617,0.034759,3.764167,-1.473139e-06,0.010152,0.000547,451,0.047464,503,0.052936,496,0.052200
4,train,01020080801023537.plt,15,2180,521.381432,0.026488,0.039892,0.681667,7.370202e-06,0.008672,0.000665,10,0.004587,1,0.000459,21,0.009633
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3672,bike,17920081115105653.plt,13,521,4.944846,0.002576,0.023770,0.313889,1.760816e-05,0.001693,0.000715,225,0.431862,4,0.007678,233,0.447217
3673,walk,17920081116065153.plt,7,325,10.130223,0.001389,0.006847,0.180000,1.030992e-06,0.000795,0.000385,133,0.409231,8,0.024615,165,0.507692
3674,walk,17920081117025429.plt,2,197,0.727775,0.001429,0.005870,0.109444,3.954695e-06,0.000805,0.000429,76,0.385787,3,0.015228,93,0.472081
3675,walk,17920081117030421.plt,7,243,0.775003,0.001488,0.004844,0.137778,2.049320e-06,0.000803,0.000442,104,0.427984,12,0.049383,138,0.567901


Encode labels

In [7]:
encoder = LabelEncoder()
feature_df['label'] = encoder.fit_transform(feature_df['label'])  # Encodes labels into numbers

Train/Test

In [8]:
# Define features and target, drop trajectory column
X = feature_df.drop(columns=['label', 'trajectory'])  # Replace 'label' with actual target column name
y = feature_df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=36, stratify=y)
#with straitify: class distribution stays the same on train/test

Deal with class imbalance?

In [25]:
#??

Model

In [10]:
xgb_model = xgb.XGBClassifier(
    objective="multi:softmax", 
    num_class=len(set(y)),  # Set the number of classes for multi-class
    eval_metric="mlogloss",  # Use "logloss" for binary classification
    use_label_encoder=False,
    #scale_pos_weight=scale_pos_weight,  # Adjust weight for imbalance
    n_estimators=100,  # Number of trees
    learning_rate=0.1,
    max_depth=6,
    random_state=42
)






Train the model

In [11]:
# Train model
xgb_model.fit(X_train, y_train)

/opt/anaconda3/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [17:08:50] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.1, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=6,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=100,
              n_jobs=None, num_class=5, num_parallel_tree=None, ...)

Predict the testset

In [12]:
y_pred = xgb_model.predict(X_test)

Evaluate

In [13]:
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))


Accuracy: 0.875
Classification Report:
               precision    recall  f1-score   support

           0       0.90      0.86      0.88       160
           1       0.78      0.70      0.74        83
           2       0.86      0.86      0.86       126
           3       0.79      0.63      0.70        43
           4       0.90      0.97      0.93       324

    accuracy                           0.88       736
   macro avg       0.85      0.80      0.82       736
weighted avg       0.87      0.88      0.87       736

Confusion Matrix:
 [[137   4   0   1  18]
 [  6  58  10   0   9]
 [  1   9 108   4   4]
 [  1   2   8  27   5]
 [  7   1   0   2 314]]


# Tune Hyperparamters using gridsearch

Set parmaters to search

In [14]:
param_grid = {
    "n_estimators": [100, 200, 300],  # Number of boosting rounds
    "learning_rate": [0.01, 0.1, 0.2],  # Lower values prevent overfitting
    "max_depth": [4, 6, 8],  # Complexity of trees
    "min_child_weight": [1, 3, 5],  # Minimum sum of weights in a child node
    "subsample": [0.8, 1.0],  # Fraction of training samples used
    "colsample_bytree": [0.8, 1.0]  # Fraction of features used per tree
}

In [15]:
xgb_model = xgb.XGBClassifier(
    objective="multi:softmax", 
    num_class=len(set(y)),  # Set the number of classes for multi-class
    eval_metric="mlogloss",  # Use "logloss" for binary classification
    use_label_encoder=False,
    #scale_pos_weight=scale_pos_weight,  # Adjust weight for imbalance
    n_estimators=100,  # Number of trees
    learning_rate=0.1,
    max_depth=6,
    random_state=42
)

In [16]:
# Set up GridSearchCV
grid_search = GridSearchCV(
    estimator=xgb_model,
    param_grid=param_grid,
    scoring="accuracy",
    cv=3,  # 3-fold cross-validation
    verbose=2,
    n_jobs=-1  # Use all available CPUs
)

In [17]:
# Perform GridSearchCV
grid_search.fit(X_train, y_train)

Fitting 3 folds for each of 324 candidates, totalling 972 fits


/opt/anaconda3/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [17:09:04] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/opt/anaconda3/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [17:09:04] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/opt/anaconda3/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [17:09:04] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/opt/anaconda3/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [17:09:04] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/opt/anaconda3/lib/python3.12/site-packa

[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=4, min_child_weight=1, n_estimators=100, subsample=0.8; total time=   0.7s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=4, min_child_weight=1, n_estimators=100, subsample=0.8; total time=   0.7s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=4, min_child_weight=1, n_estimators=100, subsample=1.0; total time=   0.7s


/opt/anaconda3/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [17:09:05] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/opt/anaconda3/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [17:09:05] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/opt/anaconda3/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [17:09:05] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/opt/anaconda3/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [17:09:05] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/opt/anaconda3/lib/python3.12/site-packa

[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=4, min_child_weight=1, n_estimators=100, subsample=1.0; total time=   0.6s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=4, min_child_weight=1, n_estimators=100, subsample=1.0; total time=   0.6s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=4, min_child_weight=1, n_estimators=100, subsample=0.8; total time=   0.7s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=4, min_child_weight=1, n_estimators=200, subsample=0.8; total time=   1.1s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=4, min_child_weight=1, n_estimators=200, subsample=0.8; total time=   1.1s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=4, min_child_weight=1, n_estimators=200, subsample=0.8; total time=   1.0s


...

[CV] END colsample_bytree=1.0, learning_rate=0.2, max_depth=8, min_child_weight=5, n_estimators=200, subsample=0.8; total time=   0.8s
[CV] END colsample_bytree=1.0, learning_rate=0.2, max_depth=8, min_child_weight=5, n_estimators=200, subsample=0.8; total time=   0.8s
[CV] END colsample_bytree=1.0, learning_rate=0.2, max_depth=8, min_child_weight=5, n_estimators=200, subsample=1.0; total time=   0.9s
[CV] END colsample_bytree=1.0, learning_rate=0.2, max_depth=8, min_child_weight=5, n_estimators=200, subsample=1.0; total time=   0.8s
[CV] END colsample_bytree=1.0, learning_rate=0.2, max_depth=8, min_child_weight=5, n_estimators=200, subsample=1.0; total time=   0.8s


/opt/anaconda3/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [17:11:48] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/opt/anaconda3/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [17:11:48] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/opt/anaconda3/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [17:11:48] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


[CV] END colsample_bytree=1.0, learning_rate=0.2, max_depth=8, min_child_weight=5, n_estimators=300, subsample=0.8; total time=   1.0s
[CV] END colsample_bytree=1.0, learning_rate=0.2, max_depth=8, min_child_weight=5, n_estimators=300, subsample=0.8; total time=   0.9s
[CV] END colsample_bytree=1.0, learning_rate=0.2, max_depth=8, min_child_weight=5, n_estimators=300, subsample=0.8; total time=   0.9s
[CV] END colsample_bytree=1.0, learning_rate=0.2, max_depth=8, min_child_weight=5, n_estimators=300, subsample=1.0; total time=   0.8s
[CV] END colsample_bytree=1.0, learning_rate=0.2, max_depth=8, min_child_weight=5, n_estimators=300, subsample=1.0; total time=   0.8s
[CV] END colsample_bytree=1.0, learning_rate=0.2, max_depth=8, min_child_weight=5, n_estimators=300, subsample=1.0; total time=   0.7s


/opt/anaconda3/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [17:11:49] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


GridSearchCV(cv=3,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False,
                                     eval_metric='mlogloss', feature_types=None,
                                     gamma=None, grow_policy=None,
                                     importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate...
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=100,
                                     n_jobs=None, num_class=5,
                                     num_parallel_tree=None, ...),
             n_jobs=-1,
             param_grid={'colsample_bytree': [0.8, 1.0],
                         'learning_rate': [0.01, 0.1, 0.2],
                         'max_depth': [4, 6, 8], 'min_child_weight': [1, 3, 5],
                         'n_estimators': [100, 200, 300],
                         'subsample': [0.8, 1.0]},
             scoring='accuracy', verbose=2)

In [18]:
# Print the best parameters
print("Best Parameters:", grid_search.best_params_)
print("Best Cross-Validation Accuracy:", grid_search.best_score_)

Best Parameters: {'colsample_bytree': 0.8, 'learning_rate': 0.2, 'max_depth': 4, 'min_child_weight': 1, 'n_estimators': 300, 'subsample': 0.8}
Best Cross-Validation Accuracy: 0.8820109287343888


In [19]:
best_params = grid_search.best_params_

best_xgb_model = xgb.XGBClassifier(
    objective="multi:softmax",  
    num_class=len(set(y_train)),  
    eval_metric="mlogloss",  
    use_label_encoder=False,
    random_state=42,
    **best_params  # Use best hyperparameters
)

best_xgb_model.fit(X_train, y_train)

# Predictions
y_pred = best_xgb_model.predict(X_test)


# Decode labels back to original transport mode names
y_test = encoder.inverse_transform(y_test)
y_pred = encoder.inverse_transform(y_pred)

# Evaluate model
print("Final Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))


/opt/anaconda3/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [17:11:50] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Final Accuracy: 0.8804347826086957
Classification Report:
               precision    recall  f1-score   support

        bike       0.92      0.84      0.88       160
         bus       0.79      0.76      0.77        83
         car       0.85      0.87      0.86       126
       train       0.90      0.60      0.72        43
        walk       0.89      0.97      0.93       324

    accuracy                           0.88       736
   macro avg       0.87      0.81      0.83       736
weighted avg       0.88      0.88      0.88       736

Confusion Matrix:
 [[135   4   2   0  19]
 [  4  63   7   0   9]
 [  2   9 110   1   4]
 [  0   3   9  26   5]
 [  5   1   2   2 314]]


Use the large dataset

In [20]:
df = data_processing.read_all_users('geolife_trajectories')

[1/182] processing user 135
[2/182] processing user 132
[3/182] processing user 104
[4/182] processing user 103
[5/182] processing user 168
[6/182] processing user 157
[7/182] processing user 150
[8/182] processing user 159
[9/182] processing user 166
[10/182] processing user 161
[11/182] processing user 102
[12/182] processing user 105
[13/182] processing user 133
[14/182] processing user 134
[15/182] processing user 160
[16/182] processing user 158
[17/182] processing user 167
[18/182] processing user 151
[19/182] processing user 169
[20/182] processing user 156
[21/182] processing user 024
[22/182] processing user 023
[23/182] processing user 015
[24/182] processing user 012
[25/182] processing user 079
[26/182] processing user 046
[27/182] processing user 041
[28/182] processing user 048
[29/182] processing user 077
[30/182] processing user 083
[31/182] processing user 084
[32/182] processing user 070
[33/182] processing user 013
[34/182] processing user 014
[35/182] processing use

In [21]:
df = data_processing.process_classes(df)
df = data_processing.drop_unlabelled(df)

In [23]:
df = data_processing.calculations(df)
df = data_processing.create_segments(df)
feature_df = df.groupby('segment').apply(feature_extraction.extract_features).reset_index(drop=True)

/var/folders/6q/9kdss80974ldrx7hy_83wyn00000gn/T/ipykernel_35050/717751794.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  feature_df = df.groupby('segment').apply(feature_extraction.extract_features).reset_index(drop=True)


In [24]:
feature_df

,label,trajectory,start_time,trajectory_length,total_distance,average_speed,max_speed,duration,average_acceleration,speed_std,acceleration_std,bearing_changes,HCR,stops,SR,VC,VCR
0,bike,20070412101853.plt,10,2,0.013280,0.000511,0.000511,0.007222,NaN,NaN,NaN,0,0.000000,0,0.000000,0,0.000000
1,walk,20070412101853.plt,10,1,0.015877,0.000194,0.000194,0.000000,-0.000004,NaN,NaN,0,0.000000,1,1.000000,1,1.000000
2,bike,20070412102116.plt,10,54,2.744276,0.002698,0.009508,4.593333,0.000127,0.001779,0.000701,23,0.425926,7,0.129630,26,0.481481
3,walk,20070412102325.plt,10,3,1.587126,0.000212,0.000511,0.027222,0.000005,0.000303,0.000005,2,0.666667,2,0.666667,2,0.666667
4,bike,20070412134621.plt,13,27,2.711323,0.003505,0.004522,0.813611,0.000006,0.001307,0.000026,8,0.296296,3,0.111111,4,0.148148
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17665,bus,20111231075505.plt,7,123,2.985901,0.002529,0.012581,0.155556,0.000015,0.003291,0.000378,73,0.593496,37,0.300813,70,0.569106
17666,walk,20111231075505.plt,8,107,1.016969,0.001587,0.010779,0.155556,-0.000015,0.001942,0.000375,68,0.635514,17,0.158879,66,0.616822
17667,walk,20111231144437.plt,14,80,13.136109,0.001587,0.028927,0.188889,0.000006,0.003174,0.000897,47,0.587500,7,0.087500,38,0.475000
17668,car,20111231144437.plt,14,252,14.029307,0.011136,0.021326,0.343056,0.000006,0.006651,0.000316,45,0.178571,24,0.095238,59,0.234127


In [25]:
encoder = LabelEncoder()
feature_df['label'] = encoder.fit_transform(feature_df['label'])  # Encodes labels into numbers

In [27]:
# Define features and target, drop trajectory column
X = feature_df.drop(columns=['label', 'trajectory'])  
y = feature_df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=36, stratify=y)

In [110]:
X_train

,start_time,trajectory_length,total_distance,average_speed,max_speed,duration,average_acceleration,speed_std,acceleration_std,bearing_changes,HCR,stops,SR,VC,VCR
3619,11,229,12.389912,0.001507,0.008465,3.015278,4.726470e-07,0.000862,0.000182,118,0.515284,8,0.034934,117,0.510917
77,22,537,1440.476136,0.023637,0.698681,16.302222,6.228582e-05,0.032120,0.002090,161,0.299814,30,0.055866,192,0.357542
136,14,85,2.613036,0.002236,0.026007,0.469722,2.211730e-04,0.003454,0.001548,32,0.376471,1,0.011765,53,0.623529
4144,0,5053,8.955003,0.002624,0.035475,0.572500,1.492501e-03,0.003694,0.007058,3631,0.718583,4,0.000792,35,0.006927
941,2,1183,8.010222,0.003769,0.017448,0.385000,3.603007e-06,0.001544,0.001257,200,0.169062,21,0.017751,253,0.213863
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3475,10,217,16.667409,0.004564,0.019563,0.330556,1.422446e-04,0.004118,0.001696,106,0.488479,38,0.175115,99,0.456221
1101,16,356,4.916479,0.001802,0.006870,0.199722,1.832780e-06,0.000979,0.000523,210,0.589888,4,0.011236,196,0.550562
4055,1,14736,65.907542,0.008506,0.061640,13.165833,1.542662e-03,0.007597,0.009777,8060,0.546960,60,0.004072,82,0.005565
788,11,177,3.993803,0.003325,0.011500,0.145556,7.447984e-06,0.003153,0.000647,63,0.355932,23,0.129944,79,0.446328


Train model on best parameters of the small model

In [28]:
best_xgb_model = xgb.XGBClassifier(
    objective="multi:softmax",  
    num_class=len(set(y_train)),  
    eval_metric="mlogloss",  
    use_label_encoder=False,
    random_state=42,
    **best_params  # Use best hyperparameters
)

best_xgb_model.fit(X_train, y_train)

# Predictions
y_pred = best_xgb_model.predict(X_test)


# Decode labels back to original transport mode names
y_test = encoder.inverse_transform(y_test)
y_pred = encoder.inverse_transform(y_pred)

# Evaluate model
print("Final Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))


/opt/anaconda3/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [17:14:02] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Final Accuracy: 0.8831352574985851
Classification Report:
               precision    recall  f1-score   support

        bike       0.92      0.87      0.89       236
         bus       0.82      0.80      0.81       474
         car       0.94      0.94      0.94      1128
       train       0.83      0.94      0.88       812
        walk       0.89      0.81      0.85       884

    accuracy                           0.88      3534
   macro avg       0.88      0.87      0.87      3534
weighted avg       0.88      0.88      0.88      3534

Confusion Matrix:
 [[ 205    5    1    0   25]
 [   9  380   47    5   33]
 [   1   55 1055    9    8]
 [   2    9   17  765   19]
 [   7   12    7  142  716]]
